# Imports

In [5]:
import tensorflow as tf

import os
import tempfile
import zipfile
import tqdm
import glob

from cral.pipeline import ObjectDetectionPipe
from cral.models.object_detection import RetinanetConfig

from segmind import set_project
set_project('33db9938-c62a-4c50-a3c8-9083acaf5dd2')

# Download Chess dataset

In [6]:
def download_chess_dataset(dataset_path=tempfile.gettempdir()):
    zip_url = 'https://public.roboflow.ai/ds/uBYkFHtqpy?key=HZljsh2sXY'
    path_to_zip_file = tf.keras.utils.get_file(
        'chess_pieces.zip',
        zip_url,
        cache_dir=dataset_path, 
        cache_subdir='',
        extract=False)
    directory_to_extract_to = os.path.join(dataset_path,'chess_pieces')
    with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
        zip_ref.extractall(directory_to_extract_to)

    images_dir = os.path.join(dataset_path, 'chess_pieces','train')
    annotation_dir = os.path.join(dataset_path, 'chess_pieces','train')

    for image in tqdm.tqdm(glob.glob(os.path.join(images_dir, '*.jpg'))):
        new_name = image.replace('_jpg.rf.', '')
        os.rename(image, new_name)

        annotation = image.replace('.jpg', '.xml')
        new_name = annotation.replace('_jpg.rf.', '')
        os.rename(annotation, new_name)

    return images_dir, annotation_dir

In [7]:
images_dir, annotation_dir = download_chess_dataset('./')

100%|██████████| 202/202 [00:00<00:00, 30631.96it/s]


# SetUp pipeline

In [8]:
pipe = ObjectDetectionPipe()

pipe.add_data(
    train_images_dir=images_dir,
    train_anno_dir=annotation_dir,
    annotation_format='pascal_voc',
    split=0.2)

pipe.lock_data()

pipe.set_algo(feature_extractor='resnet50', config=RetinanetConfig())

100%|███████████████████████████████████████████████████████████| 202/202 [00:00<00:00, 1144.87it/s]



Processing Training Dataset



100%|█████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 1320.87it/s]


Creating train Tfrecords

Creating test Tfrecords


# Train

In [9]:
from segmind.keras import CheckpointCallback, KerasCallback

keras_cb = KerasCallback()
ckpt_cb = CheckpointCallback(
            snapshot_interval=10,
            snapshot_path='/tmp',
            checkpoint_prefix='retinanet_test')
pipe.train(
    num_epochs=10,
    snapshot_prefix='test_retinanet',
    snapshot_path='/tmp',
    snapshot_every_n=10,
    batch_size=4,
    #steps_per_epoch=100,
    callbacks=[keras_cb, ckpt_cb])

Epoch 1/10


2020/11/25 12:24:44 INFO segmind.store.tracking.rest_store: Artifact upload HTTP status code: 204


41/40 [==============================] - 32s 778ms/step - loss: 2.8985 - regression_loss: 2.1201 - classification_loss: 0.7784 - val_loss: 1.8645 - val_regression_loss: 1.4100 - val_classification_loss: 0.4545
Epoch 2/10
41/40 [==============================] - 31s 748ms/step - loss: 1.5387 - regression_loss: 1.1480 - classification_loss: 0.3907 - val_loss: 1.3989 - val_regression_loss: 1.0997 - val_classification_loss: 0.2992
Epoch 3/10
41/40 [==============================] - 31s 753ms/step - loss: 1.1994 - regression_loss: 0.9295 - classification_loss: 0.2698 - val_loss: 1.0659 - val_regression_loss: 0.8343 - val_classification_loss: 0.2316
Epoch 4/10
41/40 [==============================] - 31s 758ms/step - loss: 0.9583 - regression_loss: 0.7694 - classification_loss: 0.1889 - val_loss: 0.9293 - val_regression_loss: 0.7523 - val_classification_loss: 0.1770
Epoch 5/10
41/40 [==============================] - 31s 764ms/step - loss: 0.8236 - regression_loss: 0.6852 - classification_lo

2020/11/25 12:30:55 INFO segmind.store.tracking.rest_store: Artifact upload HTTP status code: 204


# Inference

## Imports

In [14]:
from cral.models.object_detection import annotate_image
from PIL import Image
import numpy as np

## Load checkpoint files

In [11]:
checkpoint_file= '/tmp/test_retinanet_final'
prediction_func= pipe.prediction_model(checkpoint_file= checkpoint_file)

converting ...


## Predict and annotate Image

In [16]:
img_path= 'chess_pieces/test/5a35ba2ec3e0d0b2b12b1758a8ac29aa_jpg.rf.280f9940defacbb5d840aef65a9257e5.jpg'

# make prediction
bboxes, scores, labels= prediction_func(img_path)

# draw predictions on image and save
image_array= np.array(Image.open(img_path))
drawn_image= annotate_image(image_array, bboxes, scores, labels, threshold=0.5)
drawn_image.save('image_retinanet.jpeg')